In [107]:
from selenium import webdriver 
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import numpy as np

In [5]:
link='https://stockmages.netlify.app'
browser=webdriver.Chrome()
browser.get(link)
for i in tqdm(range(0,5000000,1000)):
    browser.execute_script(f"window.scrollTo(0, {i});")
    time.sleep(.1)


100%|██████████████████████████████████████████████████████████████████████████████| 5000/5000 [13:50<00:00,  6.02it/s]


In [18]:
soup=BeautifulSoup(browser.page_source,'html.parser')

In [80]:
data=[]
for sp in tqdm(soup.find_all('div',class_='container')):
    image_link=sp.find(class_='source-img').get('src')
    tags=','.join(list(set(sp.find(class_="tag-color").text[7:].split(','))))
    likes_comments=sp.find(class_='likes-comments').find_all('span')
    likes=likes_comments[0].text[:-6]
    comments=likes_comments[1].text[:-9]
    data.append([image_link,tags,likes,comments])

100%|████████████████████████████████████████████████████████████████████████████| 9104/9104 [00:03<00:00, 2299.16it/s]


In [90]:
df=pd.DataFrame(data,columns=['image_link','tags','likes','comments'])
df.head()

,image_link,tags,likes,comments
0,https://cdn.pixabay.com/photo/2022/03/06/05/30...,"Sky, Blue Sky,Clouds, Atmosphere",196,55
1,https://cdn.pixabay.com/photo/2022/04/07/11/45...,"Bird, Hummingbird, Ornithology",76,20
2,https://cdn.pixabay.com/photo/2022/02/28/15/28...,"Subtropical, Rainbow,Sea, Rainfall",282,106
3,https://cdn.pixabay.com/photo/2022/04/04/02/52...,"Japan, Sakura,Cherry Blossoms, Road",42,11
4,https://cdn.pixabay.com/photo/2022/04/09/18/06...,"Flower,Cape Marguerite, Plant",39,15


In [109]:

def download(link,image_path):
     try:
        res=requests.get(link, timeout=10)
        fd=open(image_path, 'wb')
        fd.write(res.content)
        fd.close()
        return True
     except:
        return False
image_paths=[]
image_ids=[]
i=1
for link in tqdm(df['image_link']):
    image_id='_' + str(i) + '_'
    i+=1
    image_path = f"D:/web_scarpping_projects/image_scrapping_downloads/{image_id}.jpg"
    success=download(link, image_path)
    if success:
        image_paths.append(image_path)
        image_ids.append(image_id)
    else:
        image_paths.append(np.nan)
        image_ids.append(np.nan)
df['image_ids'] = image_ids
df['image_paths'] = image_paths
df.to_csv("bulk_images_scrapping.csv")


100%|██████████████████████████████████████████████████████████████████████████████| 9104/9104 [58:29<00:00,  2.59it/s]


In [111]:
df.head()

,image_link,tags,likes,comments,image_ids,image_paths
0,https://cdn.pixabay.com/photo/2022/03/06/05/30...,"Sky, Blue Sky,Clouds, Atmosphere",196,55,_1_,D:/web_scarpping_projects/image_scrapping_down...
1,https://cdn.pixabay.com/photo/2022/04/07/11/45...,"Bird, Hummingbird, Ornithology",76,20,_2_,D:/web_scarpping_projects/image_scrapping_down...
2,https://cdn.pixabay.com/photo/2022/02/28/15/28...,"Subtropical, Rainbow,Sea, Rainfall",282,106,NaN,NaN
3,https://cdn.pixabay.com/photo/2022/04/04/02/52...,"Japan, Sakura,Cherry Blossoms, Road",42,11,NaN,NaN
4,https://cdn.pixabay.com/photo/2022/04/09/18/06...,"Flower,Cape Marguerite, Plant",39,15,_5_,D:/web_scarpping_projects/image_scrapping_down...
